# 1141_社會資料分析 加分題
學生：劉晏成

In [1]:
from load import load_sav
import pandas as pd

df = load_sav("../data/brilliant_working.sav")

In [2]:
variable_value_labels = df.attrs['variable_value_labels']
column_names = df.attrs['column_names']
column_names_to_labels = df.attrs['column_names_to_labels']

## Q1: 按照下列規則，將「配偶的教育程度」 （a10）重新編碼成「配偶教育年數」 （s_eduy）  後，報告「配偶教育年數」的平均數和標準差。

| 教育程度類別                 | 教育年數 |
|------------------------|------|
| 無、自修                   | 0    |
| 小學肄業                   | 6    |
| 國（初）中、初職               | 9    |
| 高中普通科、高中職業科、高職、士官學校    | 12   |
| 五專、二專、軍警專修班、軍警專科班、空中行專 | 14   |
| 三專                     | 15   |
| 空中大學、軍警官學校、技術學院、科大、大學  | 16   |
| 碩士                     | 19   |
| 博士                     | 24   |


In [3]:
df_1 = df[['a10']].copy()

In [24]:
spouse_education_variable_value_labels = variable_value_labels['a10']

依據編碼表編碼並預覽類別數目

In [5]:
import numpy as np

def categorize_education(value):
    category_mapping = {
        1: 0,
        2: 0,
        3: 6,
        4: 9,
        5: 9,
        6: 12,
        7: 12,
        8: 12,
        9: 12,
        10: 14,
        11: 14,
        13: 14,
        14: 14,
        15: 14,
        12: 15,
        16: 16,
        17: 16,
        18: 16,
        19: 16,
        20: 19,
        21: 24
    }
    
    return category_mapping.get(value, np.nan)

df_1_recoded = df_1.copy()
df_1_recoded.loc[:, 's_eduy'] = df_1_recoded['a10'].apply(categorize_education)

df_1_recoded['s_eduy'].value_counts().sort_index()

s_eduy
0.0       8
6.0      55
9.0      88
12.0    204
14.0     80
15.0      6
16.0    145
19.0     31
24.0      7
Name: count, dtype: int64

In [6]:
df_1_describe = df_1_recoded['s_eduy'].describe()

mean = df_1_describe['mean']
std = df_1_describe['std']
median = df_1_describe['50%']
_min = df_1_describe['min']
_max = df_1_describe['max']
q1 = df_1_describe['25%']
q3 = df_1_describe['75%']
mode = df_1_recoded.loc[:, 's_eduy'].mode()[0]
interquartile_range = q3 - q1

計算敘述性統計

In [7]:
import pandas as pd

answer = pd.DataFrame({
    'Value': [mean, median, mode, _max - _min, q1, q3, interquartile_range, std]
}, index = ['Mean', 'Median', 'Mode', 'Range', 'Q1', 'Q3', 'Interquartile Range', 'Standard Deviation'])
answer

,Value
Mean,12.591346
Median,12.000000
Mode,12.000000
Range,24.000000
Q1,12.000000
Q3,16.000000
Interquartile Range,4.000000
Standard Deviation,3.785330


### Answer: 

配偶教育年數的平均數和標準差分別為12.59 與3.79

## Q2 比較不同性別者（a1）的配偶教育年數是否有顯著差異？
1. 列出不同性別者的配偶教育年數平均分數
2. 按照假設檢定四步驟，做獨立樣本 t 檢定

In [8]:
df_2 = new_df = pd.concat([df['a1'], df_1_recoded['s_eduy']], axis=1).copy()

df_2

,a1,s_eduy
0,2.0,16.0
1,1.0,12.0
2,2.0,NaN
3,1.0,NaN
4,2.0,16.0
...,...,...
863,2.0,6.0
864,2.0,12.0
865,2.0,9.0
866,1.0,NaN


檢視遺漏值狀況

In [9]:
from statistic.missing_value import check_missing_value

check_missing_value(df_2)

,# of Missing Values
a1,0
s_eduy,244


刪去遺漏值資料

In [10]:
df_2_cleared = df_2.dropna()

In [11]:
from statistic.missing_value import check_missing_value

check_missing_value(df_2_cleared)

,# of Missing Values
a1,0
s_eduy,0


計算不同性別其配偶教育年數的敘述性統計

In [12]:
df_2_male = df_2_cleared.loc[df_2_cleared['a1'] == 1.0, 's_eduy']
df_2_female = df_2_cleared.loc[df_2_cleared['a1'] == 2.0, 's_eduy']

df_2_raw = pd.DataFrame({'Male': df_2_male, 'Female': df_2_female})
df_2_raw.describe()

,Male,Female
count,282.000000,342.000000
mean,12.351064,12.789474
std,3.829203,3.742771
min,0.000000,0.000000
25%,9.750000,12.000000
50%,12.000000,12.000000
75%,16.000000,16.000000
max,24.000000,24.000000


接著針對男性的配偶教育年數與女性的配偶教育年數執行獨立樣本T 檢定

首先，建立檢定假設以及判斷基準

男性、女性配偶教育年數平均數為 $\mu_1$、$\mu_2$  
$H_0: \mu_1 = \mu_2$  
$H_1: \mu_1 \neq \mu_2$  
$\alpha = 0.05$

執行獨立樣本T 檢定

In [13]:
from statistic import t_test

t_test.compare_independent_samples(df_2_male, df_2_female) 

,Statistic,p-value
Levene's Test for Equal Variances,0.261063,0.609572
t-test (Equal Variance Assumed),-1.441109,0.150057
t-test (Equal Variance Not Assumed),-1.437945,0.150976


**Levene's Test**  
H0: 兩母群體變異數相同  
H1: 兩母群體變異數不相同  
$\alpha = 0.05$

檢視表格中Levene's Test 的P Value > 0.05，接受Levenu's test 的H0（兩母群體變異數相等）。

接下觀察母群體變異數相同的T Test 結果：由於P Value > 0.05，接受虛無假設，不同性別之其配偶教育年數無顯著差異。

### Answer
1. 男性配偶平均教育年數為12.35；女性配偶平均教育年數為12.79
2. 男性與女性的配偶教育年數並無統計顯著差異

## Q3: 比較不同年齡分組者（agegroup）的配偶教育年數是否有顯著差異？
1. 列出不同年齡分組者的配偶教育年數平均分數
2. 按照假設檢定四步驟，做變異數分析的整體檢定
3. 如果變異數分析的整體檢定達到顯著水準，接著做事後比較

In [14]:
df_3 = pd.concat([df['agegroup'], df_1_recoded['s_eduy']], axis=1).copy()

In [15]:
from statistic.missing_value import check_missing_value

check_missing_value(df_3)

,# of Missing Values
agegroup,0
s_eduy,244


處理遺漏值數據

In [16]:
df_3_cleared = df_3.dropna()

In [17]:
check_missing_value(df_3_cleared)

,# of Missing Values
agegroup,0
s_eduy,0


首先，計算不同組別的敘述性統計

In [19]:
df_3_cleared.groupby('agegroup')['s_eduy'].describe()

,count,mean,std,min,25%,50%,75%,max
agegroup,,,,,,,,
1.0,166.0,14.301205,2.636503,0.0,12.0,16.0,16.0,19.0
2.0,269.0,13.330855,3.296059,6.0,12.0,12.0,16.0,24.0
3.0,189.0,10.037037,3.996502,0.0,6.0,9.0,12.0,24.0


青壯年組別的配偶教育平均年數為14.30；中年組的配偶教育平均年數為13.33；老年組的配偶教育平均年數為10.04

考察資料，確認不同年齡組別分布

In [20]:
df_3_cleared['agegroup'].value_counts()

agegroup
2.0    269
3.0    189
1.0    166
Name: count, dtype: int64

執行ANOVA  
設定：不同年齡組別的配偶教育年數平均分別為 $\mu_1$、$\mu_2$、$\mu_3$  
$H_0: \mu_1 = \mu_2 = \mu_3$  
$H_1: \mu_k 不完全相等$  
$\alpha = 0.05$

In [21]:
from statistic.anova import perform_anova


perform_anova(df=df_3_cleared, val_col='s_eduy', group_col='agegroup')

,sum_sq,df,F,PR(>F)
Intercept,33951.060241,1.0,2985.824745,2.059310e-239
C(agegroup),1865.558866,2.0,82.033253,2.434947e-32
Residual,7061.234403,621.0,NaN,NaN


依據ANOVA 結果，agegroup 的P value < 0.05 ，拒絕虛無假設。結果顯示三組間存在差異

接下來執行Levene's test 來判斷均值性狀況，以執行事後比較

In [22]:
from scipy.stats import levene

groups = {name: group['s_eduy'].values for name, group in df_3_cleared.groupby('agegroup')}

levene_stat, levene_p = levene(*groups.values())
pd.DataFrame({ 'Values': [f"{levene_stat:.2f}", levene_p] }, index = ['Levene Stat', 'P Value'])

,Values
Levene Stat,7.85
P Value,0.00043


結果顯示P Value 遠小於0.05：非均值。  
由於樣本數>50 (Shingala and Rajyaguru, 2015)，選擇games-howell 方法求得三組間兩兩比較的結果

In [23]:
from statistic.anova import perform_post_hoc

perform_post_hoc(df_3_cleared, levene_p = levene_p, val_col='s_eduy', group_col='agegroup')

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,1.0,2.0,14.301205,13.330855,0.970350,0.286812,3.383228,404.882672,0.002264,0.316400
1,1.0,3.0,14.301205,10.037037,4.264168,0.355503,11.994741,328.555698,0.000000,1.241052
2,2.0,3.0,13.330855,10.037037,3.293818,0.353404,9.320250,353.925009,0.000000,0.913094


由於事後比較的三組比較P Value 皆小於0.05 ，故三組比較關係皆具統計意義。可知青壯年組 > 中年組 > 老年組

列出不同年齡分組者的配偶教育年數平均分數
按照假設檢定四步驟，做變異數分析的整體檢定
如果變異數分析的整體檢定達到顯著水準，接著做事後比較

### Answer
1. 青壯年組別的配偶教育平均年數為14.30；中年組的配偶教育平均年數為13.33；老年組的配偶教育平均年數為10.04
2. 三組年齡組別的配偶教育平均年數不完全相同
3. 年齡組別與配偶教育年數關係為：青壯年組 > 中年組 > 老年組